In [2]:
# ==============================
# 1. IMPORTAR LIBRERÍAS
# ==============================
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, roc_curve

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import matplotlib.pyplot as plt
import seaborn as sns


In [12]:
#CARGAR DF
df = pd.read_csv('df_processed_final.csv')

#imprimir columnas
df.columns


Index(['Genero', 'Campesino', 'Estrato', 'NIvel_educacion', 'Discapacidad',
       'Tipo_formacion', 'Victima_del_conflicto', 'tiempo_segundos',
       'Puntaje_eje_tematico_selecionado', 'area_1_alfabetizacion_datos',
       'area_2_comunicacion_y_colaboracion', 'area_3_contenidos_digitales',
       'area_4_seguridad', 'area_5_solucion_de_problemas', 'area_6_igles',
       'Asistencia horas', 'Total_horas posibles', 'etnica_Indígena',
       'etnica_Negro, Mulato, Afrodescendiente, Afrocolombiano',
       'etnica_Ningún grupo étnico',
       'etnica_Raizal del Archipiélago de San Andrés, Providencia y Santa Catalina',
       'etnica_Rrom o gitano', 'linea_1_Explorador - Básico',
       'linea_1_Innovador - Avanzado', 'linea_1_Integrador - Intermedio',
       'linea_2_Explorador - Básico', 'linea_2_Innovador - Avanzado',
       'linea_2_Integrador - Intermedio', 'linea_3_Explorador - Básico',
       'linea_3_Innovador - Avanzado', 'linea_3_Integrador - Intermedio',
       'linea_4_Expl

In [21]:
# Variable objetivo
y = df["exito_academico"]

# Features (todas excepto la target)
X = df.drop(columns=[
    'exito_academico',
    'Asistencia horas',
    'Cohorte_1', 'Cohorte_2', 'Cohorte_3', 'Cohorte_4',
    'Cohorte_5', 'Cohorte_6', 'Cohorte_7', 'Cohorte_8',
    'Cohorte_2_3', 'Cohorte_3_5', 'Cohorte_4_5', 'Cohorte_5_7',
    'Cohorte_6_7', 'Cohorte_6_8',
    'dia', 'mes',
    'Tipo_de_formacion_Híbrida',

    # Variables de áreas (redundantes por promedio_areas)
    'area_1_alfabetizacion_datos',
    'area_2_comunicacion_y_colaboracion',
    'area_3_contenidos_digitales',
    'area_4_seguridad',
    'area_5_solucion_de_problemas',
    'area_6_igles',

    # Variables categóricas nivel Explorador / Innovador / Integrador
    'area_1_Explorador', 'area_1_Innovador', 'area_1_Integrador',
    'area_2_Explorador', 'area_2_Innovador', 'area_2_Integrador',
    'area_3_Explorador', 'area_3_Innovador', 'area_3_Integrador',
    'area_4_Explorador', 'area_4_Innovador', 'area_4_Integrador',
    'area_5_Explorador', 'area_5_Innovador', 'area_5_Integrador',
    
     # Eje programa (duplicado de eje final)
    'Programa_de_Formación_Arquitectura en la nube',
    'Programa_de_Formación_Blockchain',
    'Programa_de_Formación_Inteligencia artificial',
    'Programa_de_Formación_Programación',
    
    #evitar multicol
    'eje_final_Blockchain', 'Nivel_Intermedio',
    
    'eje_tematico_Blockchain',
    'eje_tematico_Ciber seguridad y Blockchain',
    'eje_tematico_Computación y arquitectura en la nube',
    'eje_tematico_Datos', 'eje_tematico_Inteligencia artificial',
    'eje_tematico_Programación', 'Programa_de_Formación_Análisis de datos'
])


In [22]:
X.columns

Index(['Genero', 'Campesino', 'Estrato', 'NIvel_educacion', 'Discapacidad',
       'Tipo_formacion', 'Victima_del_conflicto', 'tiempo_segundos',
       'Puntaje_eje_tematico_selecionado', 'Total_horas posibles',
       'etnica_Indígena',
       'etnica_Negro, Mulato, Afrodescendiente, Afrocolombiano',
       'etnica_Ningún grupo étnico',
       'etnica_Raizal del Archipiélago de San Andrés, Providencia y Santa Catalina',
       'etnica_Rrom o gitano', 'linea_1_Explorador - Básico',
       'linea_1_Innovador - Avanzado', 'linea_1_Integrador - Intermedio',
       'linea_2_Explorador - Básico', 'linea_2_Innovador - Avanzado',
       'linea_2_Integrador - Intermedio', 'linea_3_Explorador - Básico',
       'linea_3_Innovador - Avanzado', 'linea_3_Integrador - Intermedio',
       'linea_4_Explorador - Básico', 'linea_4_Innovador - Avanzado',
       'linea_4_Integrador - Intermedio', 'linea_5_Explorador - Básico',
       'linea_5_Innovador - Avanzado', 'linea_5_Integrador - Intermedio',
     

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Logistic regression

In [26]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import pandas as pd

# Imputador
imputer = SimpleImputer(strategy='mean')  # reemplaza NaN por la media

# Hiperparámetros
C_values = [0.01, 0.1, 1, 10]
penalties = ['l1', 'l2']
max_iters = [100, 500, 1000]  # distintos max_iter a probar
solver_map = {'l1': 'saga', 'l2': 'lbfgs'}


results = []

for penalty in penalties:
    for C in C_values:
        for max_iter in max_iters:
            # Imputar datos
            X_train_imputed = imputer.fit_transform(X_train)
            X_test_imputed = imputer.transform(X_test)

            # Entrenar modelo
            log_reg = LogisticRegression(
                penalty=penalty,
                C=C,
                solver=solver_map[penalty],
                max_iter=max_iter,
                class_weight='balanced'
            )
            log_reg.fit(X_train_imputed, y_train)
            y_pred = log_reg.predict(X_test_imputed)
            y_prob = log_reg.predict_proba(X_test_imputed)[:,1]

            auc = roc_auc_score(y_test, y_prob)
            acc = accuracy_score(y_test, y_pred)
            cm = confusion_matrix(y_test, y_pred)

            results.append({
                'penalty': penalty,
                'C': C,
                'max_iter': max_iter,
                'accuracy': acc,
                'auc': auc,
                'confusion_matrix': cm
            })


# Convertir a DataFrame y ordenar por AUC
results_df = pd.DataFrame(results).sort_values(by='auc', ascending=False)
print(results_df)


/Users/sophiaaristizabal/Documents/GitHub/Proyecto_Machine_Learning/venv/lib/python3.14/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/sophiaaristizabal/Documents/GitHub/Proyecto_Machine_Learning/venv/lib/python3.14/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/sophiaaristizabal/Documents/GitHub/Proyecto_Machine_Learning/venv/lib/python3.14/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/sophiaaristizabal/Documents/GitHub/Proyecto_Machine_Learning/venv/lib/python3.14/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/sophiaaristizabal/Documents/GitHub/Proyecto_M

   penalty      C  max_iter  accuracy       auc      confusion_matrix
7       l1   1.00       500  0.991952  0.996923  [[219, 4], [0, 274]]
10      l1  10.00       500  0.991952  0.996923  [[219, 4], [0, 274]]
4       l1   0.10       500  0.991952  0.996907  [[219, 4], [0, 274]]
9       l1  10.00       100  0.989940  0.996907  [[218, 5], [0, 274]]
3       l1   0.10       100  0.989940  0.996890  [[218, 5], [0, 274]]
6       l1   1.00       100  0.989940  0.996890  [[218, 5], [0, 274]]
8       l1   1.00      1000  0.991952  0.996890  [[219, 4], [0, 274]]
11      l1  10.00      1000  0.991952  0.996890  [[219, 4], [0, 274]]
5       l1   0.10      1000  0.991952  0.996825  [[219, 4], [0, 274]]
12      l2   0.01       100  0.991952  0.996678  [[219, 4], [0, 274]]
13      l2   0.01       500  0.991952  0.996678  [[219, 4], [0, 274]]
14      l2   0.01      1000  0.991952  0.996678  [[219, 4], [0, 274]]
0       l1   0.01       100  0.989940  0.993879  [[218, 5], [0, 274]]
1       l1   0.01   

# Random Forest

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.impute import SimpleImputer
import pandas as pd

# Imputación (opcional, si hay NaN)
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Hiperparámetros a probar
n_estimators_list = [100, 200, 500]          # número de árboles
max_depth_list = [None, 5, 10, 20]           # profundidad máxima
min_samples_split_list = [2, 5, 10]          # min samples para dividir
min_samples_leaf_list = [1, 2, 4]            # min samples por hoja

results_rf = []

for n_estimators in n_estimators_list:
    for max_depth in max_depth_list:
        for min_samples_split in min_samples_split_list:
            for min_samples_leaf in min_samples_leaf_list:
                rf = RandomForestClassifier(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    min_samples_split=min_samples_split,
                    min_samples_leaf=min_samples_leaf,
                    class_weight='balanced',
                    random_state=42,
                    n_jobs=-1
                )
                rf.fit(X_train_imputed, y_train)
                y_pred = rf.predict(X_test_imputed)
                y_prob = rf.predict_proba(X_test_imputed)[:,1]

                auc = roc_auc_score(y_test, y_prob)
                acc = accuracy_score(y_test, y_pred)
                cm = confusion_matrix(y_test, y_pred)

                results_rf.append({
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'min_samples_split': min_samples_split,
                    'min_samples_leaf': min_samples_leaf,
                    'accuracy': acc,
                    'auc': auc,
                    'confusion_matrix': cm
                })

# Convertir a DataFrame y ordenar por AUC
results_rf_df = pd.DataFrame(results_rf).sort_values(by='auc', ascending=False)
print(results_rf_df)


    n_estimators  max_depth  min_samples_split  min_samples_leaf  accuracy  \
82           500        5.0                  2                 2  0.991952   
89           500        5.0                 10                 4  0.991952   
83           500        5.0                  2                 4  0.991952   
86           500        5.0                  5                 4  0.991952   
14           100        5.0                  5                 4  0.991952   
..           ...        ...                ...               ...       ...   
22           100       10.0                  5                 2  0.991952   
57           200       10.0                  5                 1  0.991952   
63           200       20.0                  2                 1  0.991952   
36           200        NaN                  2                 1  0.991952   
58           200       10.0                  5                 2  0.991952   

         auc      confusion_matrix  
82  0.997185  [[219, 4], [

# XGBOOST

In [30]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.impute import SimpleImputer
import pandas as pd

# Imputación opcional si hay NaN
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Hiperparámetros a probar
n_estimators_list = [100, 200, 500]
max_depth_list = [3, 5, 7, 10]
learning_rate_list = [0.01, 0.05, 0.1]
subsample_list = [0.7, 0.8, 1.0]

results_xgb = []

for n_estimators in n_estimators_list:
    for max_depth in max_depth_list:
        for lr in learning_rate_list:
            for subsample in subsample_list:
                model = xgb.XGBClassifier(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    learning_rate=lr,
                    subsample=subsample,
                    use_label_encoder=False,
                    eval_metric='logloss',
                    random_state=42,
                    n_jobs=-1
                )
                model.fit(X_train_imputed, y_train)
                y_pred = model.predict(X_test_imputed)
                y_prob = model.predict_proba(X_test_imputed)[:,1]

                auc = roc_auc_score(y_test, y_prob)
                acc = accuracy_score(y_test, y_pred)
                cm = confusion_matrix(y_test, y_pred)

                results_xgb.append({
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'learning_rate': lr,
                    'subsample': subsample,
                    'accuracy': acc,
                    'auc': auc,
                    'confusion_matrix': cm
                })

# Convertir a DataFrame y ordenar por AUC
results_xgb_df = pd.DataFrame(results_xgb).sort_values(by='auc', ascending=False)
print(results_xgb_df)


/Users/sophiaaristizabal/Documents/GitHub/Proyecto_Machine_Learning/venv/lib/python3.14/site-packages/xgboost/training.py:199: UserWarning: [22:34:49] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/sophiaaristizabal/Documents/GitHub/Proyecto_Machine_Learning/venv/lib/python3.14/site-packages/xgboost/training.py:199: UserWarning: [22:34:49] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/sophiaaristizabal/Documents/GitHub/Proyecto_Machine_Learning/venv/lib/python3.14/site-packages/xgboost/training.py:199: UserWarning: [22:34:49] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/sophiaaristizabal/Documents/GitHub/Proyecto_Machine_Learning

     n_estimators  max_depth  learning_rate  subsample  accuracy       auc  \
107           500         10           0.10        1.0  0.991952  0.993732   
98            500          7           0.10        1.0  0.991952  0.993732   
89            500          5           0.10        1.0  0.991952  0.993732   
80            500          3           0.10        1.0  0.991952  0.993732   
39            200          3           0.05        0.7  0.991952  0.993715   
..            ...        ...            ...        ...       ...       ...   
64            200         10           0.01        0.8  0.991952  0.991031   
54            200          7           0.01        0.7  0.991952  0.991031   
83            500          5           0.01        1.0  0.991952  0.990639   
92            500          7           0.01        1.0  0.991952  0.990639   
101           500         10           0.01        1.0  0.991952  0.990639   

         confusion_matrix  
107  [[219, 4], [0, 274]]  
98   [[